# �� Preprocesamiento Definitivo para Survival Analysis

**Versión:** MASTER (Corregida)  
**Problema Resuelto:** Duration discreta (solo 3 valores) → Imputación Estocástica Uniforme

**Fundamento Científico:**
> Lawless (2003), "Statistical Models and Methods for Lifetime Data":
> Datos de intervalo deben tratarse como continuos mediante imputación para preservar varianza.

**La Solución:**
- Convertir rangos categóricos ("Entre 6 meses y 1 año") en valores continuos
- Usar distribución uniforme: $T \sim U(lower, upper)$

---

In [1]:
# ==============================================================================
# 0. CONFIGURACIÓN Y CARGA
# ==============================================================================
import pandas as pd
import numpy as np
import json
import re
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths
DATA_RAW = Path("data/raw")
DATA_PROCESSED = Path("data/processed")
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# Ventana máxima de observación (meses desde graduación hasta encuesta)
WINDOW_SIZE = 30.0  # Asumimos encuesta hasta 30 meses post-egreso

print(f"✅ Configuración cargada")
print(f"   Random State: {RANDOM_STATE}")
print(f"   Ventana de observación: {WINDOW_SIZE} meses")

✅ Configuración cargada
   Random State: 42
   Ventana de observación: 30.0 meses


In [2]:
# Dependencias
try:
    from unidecode import unidecode
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
except ImportError:
    !pip install unidecode scikit-learn pyarrow -q
    from unidecode import unidecode
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split

# Cargar datos
df_raw = pd.read_excel(DATA_RAW / "Encuesta recien graduados - pregrado(1).xlsx")

# Cargar diccionario de competencias
with open(DATA_RAW / "diccionario_maestro_stem.json", 'r', encoding='utf-8') as f:
    DICCIONARIO_MAESTRO = json.load(f)['competencias']

print(f"✅ Datos cargados: {len(df_raw)} registros")
print(f"✅ Competencias técnicas: {len(DICCIONARIO_MAESTRO)}")

✅ Datos cargados: 380 registros
✅ Competencias técnicas: 52


---
## 1. Mapeo y Limpieza de Columnas

In [3]:
# Mapeo de columnas por índice
COLUMN_MAP = {
    0: "edad",
    1: "genero",
    11: "trabaja_actualmente",
    15: "tiempo_trabajando_cat",      # CATEGORÍA: "Menos de 6 meses", etc.
    16: "correspondencia_formacion",
    30: "busca_trabajo",
    31: "tiempo_buscando_cat",        # CATEGORÍA para desempleados
    35: "asignaturas_relevantes",
    40: "carrera",
}

HAB_BLANDAS_MAP = {
    3: "hab_gestion",
    4: "hab_comunicacion", 
    5: "hab_liderazgo",
    6: "hab_trabajo_equipo",
    7: "hab_etica",
    8: "hab_responsabilidad",
    9: "hab_aprendizaje"
}

df = pd.DataFrame()
for idx, name in {**COLUMN_MAP, **HAB_BLANDAS_MAP}.items():
    df[name] = df_raw.iloc[:, idx]

# Limpieza básica
df['trabaja_actualmente'] = df['trabaja_actualmente'].fillna('No')
df['busca_trabajo'] = df['busca_trabajo'].fillna('No')
df['correspondencia_formacion'] = pd.to_numeric(df['correspondencia_formacion'], errors='coerce').fillna(0)

print(f"✅ Columnas mapeadas: {len(df.columns)}")

✅ Columnas mapeadas: 16


---
## 2. Imputación Estocástica Uniforme (Lawless 2003)

Convertimos categorías de tiempo en valores continuos usando $T \sim U(lower, upper)$

In [4]:
# ==============================================================================
# IMPUTACIÓN DE INTERVALOS A CONTINUOS
# ==============================================================================

def parse_interval_to_range(text):
    """Convierte texto de encuesta a rango numérico [min, max] en meses."""
    if pd.isna(text):
        return np.nan, np.nan
    
    text = str(text).lower()
    
    if "menos de 6 meses" in text:
        return 0.1, 6.0
    elif "entre 6 meses y 1 año" in text or "de 6 meses a 1 año" in text:
        return 6.0, 12.0
    elif "entre 1 y 2 años" in text or "de 1 año a 2 años" in text:
        return 12.0, 24.0
    elif "más de 2 años" in text or "mas de 2 años" in text:
        return 24.0, 30.0
    
    return np.nan, np.nan

# 1. Simular Tenure Continua (para empleados)
lower_t, upper_t = zip(*df['tiempo_trabajando_cat'].apply(parse_interval_to_range))
lower_t = np.array(lower_t, dtype=float)
upper_t = np.array(upper_t, dtype=float)

# Donde hay NaN, ponemos valores neutros para evitar errores
mask_valid_t = ~np.isnan(lower_t)
tenure_sim = np.zeros(len(df))
tenure_sim[mask_valid_t] = np.random.uniform(lower_t[mask_valid_t], upper_t[mask_valid_t])
df['tenure_simulated'] = tenure_sim

# 2. Simular Tiempo de Búsqueda Continuo (para desempleados)
lower_b, upper_b = zip(*df['tiempo_buscando_cat'].apply(parse_interval_to_range))
lower_b = np.array(lower_b, dtype=float)
upper_b = np.array(upper_b, dtype=float)

mask_valid_b = ~np.isnan(lower_b)
search_sim = np.zeros(len(df))
search_sim[mask_valid_b] = np.random.uniform(lower_b[mask_valid_b], upper_b[mask_valid_b])
df['searching_simulated'] = search_sim

print("✅ Intervalos convertidos a continuos (Imputación Estocástica Uniforme)")
print(f"   Tenure simulada - valores únicos: {df['tenure_simulated'].nunique()}")
print(f"   Búsqueda simulada - valores únicos: {df['searching_simulated'].nunique()}")

✅ Intervalos convertidos a continuos (Imputación Estocástica Uniforme)
   Tenure simulada - valores únicos: 210
   Búsqueda simulada - valores únicos: 163


---
## 3. Construcción del Target de Supervivencia

**Fórmulas:**
- Evento (E=1): Trabaja + Correspondencia ≥ 3
- Duration para E=1: $T = Ventana - Tenure$ (tiempo que ESPERÓ)
- Duration para E=0: Tiempo buscando (censurado)

In [5]:
# ==============================================================================
# CONSTRUCCIÓN DE EVENT Y DURATION (CORREGIDA)
# ==============================================================================

df['event'] = 0
df['duration'] = np.nan

# -----------------------------------------------------------------------------
# CASO 1: EVENTO - Trabaja con correspondencia >= 3
# -----------------------------------------------------------------------------
mask_event = (df['trabaja_actualmente'] == 'Si') & (df['correspondencia_formacion'] >= 3)
df.loc[mask_event, 'event'] = 1

# Duration = Ventana - Tenure (cuánto ESPERÓ antes de conseguir empleo)
t_event = WINDOW_SIZE - df.loc[mask_event, 'tenure_simulated']
df.loc[mask_event, 'duration'] = t_event.clip(lower=0.1)

# -----------------------------------------------------------------------------
# CASO 2: CENSURADO - Busca trabajo activamente
# -----------------------------------------------------------------------------
mask_censored = (df['trabaja_actualmente'] == 'No') & (df['busca_trabajo'] == 'Si')
df.loc[mask_censored, 'event'] = 0
df.loc[mask_censored, 'duration'] = df.loc[mask_censored, 'searching_simulated'].clip(lower=0.1)

# -----------------------------------------------------------------------------
# CASO 3: CENSURADO - Subempleo (trabaja pero NO relacionado)
# -----------------------------------------------------------------------------
mask_underemp = (df['trabaja_actualmente'] == 'Si') & (df['correspondencia_formacion'] < 3)
df.loc[mask_underemp, 'event'] = 0
df.loc[mask_underemp, 'duration'] = WINDOW_SIZE  # Censurado al máximo

# -----------------------------------------------------------------------------
# Limpiar inactivos y NaN
# -----------------------------------------------------------------------------
df_clean = df.dropna(subset=['duration']).copy()
df_clean = df_clean[df_clean['duration'] > 0].copy()

print("=" * 60)
print("✅ TARGET CONSTRUIDO (Imputación Estocástica)")
print("=" * 60)
print(f"Eventos (E=1):     {(df_clean['event'] == 1).sum()} ({100*(df_clean['event'] == 1).mean():.1f}%)")
print(f"Censurados (E=0):  {(df_clean['event'] == 0).sum()} ({100*(df_clean['event'] == 0).mean():.1f}%)")
print(f"\n🎯 Valores únicos en duration: {df_clean['duration'].nunique()} (¡YA NO SON 3!)")
print(f"Rango duration: [{df_clean['duration'].min():.2f}, {df_clean['duration'].max():.2f}] meses")

✅ TARGET CONSTRUIDO (Imputación Estocástica)
Eventos (E=1):     169 (45.6%)
Censurados (E=0):  202 (54.4%)

🎯 Valores únicos en duration: 332 (¡YA NO SON 3!)
Rango duration: [0.18, 30.00] meses


In [6]:
# Verificar que ya no hay colinealidad perfecta
corr = df_clean[['event', 'duration']].corr().iloc[0, 1]
print(f"\n🔍 Correlación event-duration: {corr:.4f}")
print(f"   (Antes era -0.93, ahora debe ser menor en magnitud)")

# Histograma rápido
print(f"\n📊 Distribución de duration:")
print(df_clean['duration'].describe().round(2))


🔍 Correlación event-duration: 0.5105
   (Antes era -0.93, ahora debe ser menor en magnitud)

📊 Distribución de duration:
count    371.00
mean      15.54
std       11.10
min        0.18
25%        4.23
50%       17.44
75%       26.37
max       30.00
Name: duration, dtype: float64


---
## 4. Extracción de Soft Skills

In [7]:
# Extraer valores Likert
def extract_likert(value):
    if pd.isna(value):
        return np.nan
    match = re.search(r'^(\d)', str(value))
    return int(match.group(1)) if match else np.nan

HAB_COLS = list(HAB_BLANDAS_MAP.values())
for col in HAB_COLS:
    df_clean[col] = df_clean[col].apply(extract_likert)

print("✅ Soft Skills extraídas (Likert 1-5)")
print(df_clean[HAB_COLS].describe().loc[['count', 'mean']].round(2))

✅ Soft Skills extraídas (Likert 1-5)
       hab_gestion  hab_comunicacion  hab_liderazgo  hab_trabajo_equipo  \
count       371.00             371.0          371.0              371.00   
mean          3.71               3.4            3.7                3.84   

       hab_etica  hab_responsabilidad  hab_aprendizaje  
count     371.00               371.00           371.00  
mean        4.13                 3.84             4.29  


---
## 5. Vectorización de Habilidades Técnicas

In [8]:
# Matching con diccionario controlado
def match_skills(texto, diccionario):
    if pd.isna(texto):
        return []
    texto = unidecode(str(texto).lower())
    matches = []
    for skill_id, skill_info in diccionario.items():
        for alias in skill_info['aliases']:
            if unidecode(alias.lower()) in texto:
                matches.append(skill_id)
                break
    return list(set(matches))

# Crear columnas binarias
SKILLS_LIST = list(DICCIONARIO_MAESTRO.keys())
for skill in SKILLS_LIST:
    df_clean[f"tech_{skill}"] = 0

for idx, row in df_clean.iterrows():
    for skill in match_skills(row["asignaturas_relevantes"], DICCIONARIO_MAESTRO):
        df_clean.loc[idx, f"tech_{skill}"] = 1

TECH_COLS = [f"tech_{s}" for s in SKILLS_LIST]
print(f"✅ {len(TECH_COLS)} competencias técnicas vectorizadas")

✅ 52 competencias técnicas vectorizadas


---
## 6. Split Estratificado y Escalado (Sin Data Leakage)

In [9]:
# Preparar features
df_clean['genero_m'] = (df_clean['genero'].str.lower() == 'masculino').astype(int)
df_clean['edad'] = pd.to_numeric(df_clean['edad'], errors='coerce')

FEATURE_COLS = ['edad', 'genero_m'] + HAB_COLS + TECH_COLS

# Split estratificado por event
X = df_clean[FEATURE_COLS].copy()
y = df_clean[['event', 'duration']].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y['event']
)

print(f"✅ Split realizado:")
print(f"   Train: {len(X_train)} | Test: {len(X_test)}")

# Escalado SOLO fit en train
scaler = MinMaxScaler()

# Imputar NaN con mediana de train
for col in HAB_COLS + ['edad']:
    train_median = X_train[col].median()
    X_train[col] = X_train[col].fillna(train_median)
    X_test[col] = X_test[col].fillna(train_median)

# Fit en train, transform en ambos
scaler.fit(X_train[HAB_COLS])
X_train[HAB_COLS] = scaler.transform(X_train[HAB_COLS])
X_test[HAB_COLS] = scaler.transform(X_test[HAB_COLS])

print("✅ Escalado aplicado (sin data leakage)")

✅ Split realizado:
   Train: 296 | Test: 75
✅ Escalado aplicado (sin data leakage)


In [10]:
# Concatenar y guardar
train_final = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
test_final = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

train_final.to_parquet(DATA_PROCESSED / "train_survival_final.parquet", index=False)
test_final.to_parquet(DATA_PROCESSED / "test_survival_final.parquet", index=False)

print("=" * 60)
print("🚀 PREPROCESAMIENTO COMPLETADO")
print("=" * 60)
print(f"\n📁 Archivos guardados:")
print(f"   - train_survival_final.parquet ({len(train_final)} registros)")
print(f"   - test_survival_final.parquet ({len(test_final)} registros)")
print(f"\n📐 Features: {len(FEATURE_COLS)}")
print(f"🎯 Duration valores únicos: {train_final['duration'].nunique()}")
print(f"\n✅ Correlación event-duration: {train_final[['event','duration']].corr().iloc[0,1]:.4f}")
print("=" * 60)

🚀 PREPROCESAMIENTO COMPLETADO

📁 Archivos guardados:
   - train_survival_final.parquet (296 registros)
   - test_survival_final.parquet (75 registros)

📐 Features: 61
🎯 Duration valores únicos: 265

✅ Correlación event-duration: 0.5079
